<a href="https://colab.research.google.com/github/andyharless/twit_demog/blob/master/twitgen_keras_init_with_glove_new_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
compare_preds = True

TRAIN_INPUT = 'twitgen_train_201906011956.csv'
VALID_INPUT = 'twitgen_valid_201906011956.csv'
TEST_INPUT = 'twitgen_test_201906011956.csv'
EMBEDDING_DIM = 200
MAXLEN = 50

In [0]:
import tensorflow as tf
import pandas as pd
import os
import re
import keras
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
from keras.optimizers import Adam, Adagrad
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string

Using TensorFlow backend.


In [0]:
basepath = '/content/gdrive/My Drive/twitgen/'
glovefile = 'glove.twitter.27B.200d.txt.gz'

In [0]:
glovepath = basepath + glovefile
!cp '$glovepath' .

In [0]:
!gunzip $glovefile

In [0]:
!ls -l

total 2009380
drwx------ 3 root root       4096 Jun  7 13:46 gdrive
-rw------- 1 root root 2057590469 Jun  7 13:46 glove.twitter.27B.200d.txt
drwxr-xr-x 1 root root       4096 May 31 16:17 sample_data


In [0]:
df_train = pd.read_csv(basepath+TRAIN_INPUT, index_col=['id','time'], parse_dates=['time'])
df_valid = pd.read_csv(basepath+VALID_INPUT, index_col=['id','time'], parse_dates=['time'])
df_test = pd.read_csv(basepath+TEST_INPUT, index_col=['id','time'], parse_dates=['time'])
df_train.head()

,,text,male
id,time,,
1083596943807393792,2019-05-27 23:27:08+00:00,"Ah, the Flat Earth gambit.\nWell played.",True
815783987784187904,2019-05-24 15:36:01+00:00,Aw ily,False
3458239641,2019-05-24 19:00:37+00:00,I hate being sick,False
1003729876250226688,2019-05-26 12:53:00+00:00,You still didn't' do shit tho. Slow down the...,True
2360143940,2019-05-28 03:50:46+00:00,Harriet Tubman may not be on the $20 bill... f...,False


In [0]:
(df_train.text.str.split().apply(len).max(), 
 df_valid.text.str.split().apply(len).max(),
 df_test.text.str.split().apply(len).max())

(34, 30, 31)

In [0]:
# Text Normalizing function. 
# Taken from 
# https://medium.com/@sabber/classifying-yelp-review-comments-using-lstm-and-word-embeddings-part-1-eb2275e4066b
# which was taken from https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings

# This is for general text, not Twitter-specific.
# Probably would get a better classifier if we used a Python transaltion of this:
# https://nlp.stanford.edu/projects/glove/preprocess-twitter.rb
# but that is arguably outside the scope of this project

def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
   # stops = set(stopwords.words("english"))
   # text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
#    text = text.split()
#    stemmer = SnowballStemmer('english')
#    stemmed_words = [stemmer.stem(word) for word in text]
#    text = " ".join(stemmed_words)
    return text


In [0]:
def get_texts_and_labels(df):
  texts = df['text'].map(lambda x: clean_text(x)).tolist()
  texts = [t.split()[0:MAXLEN] for t in texts]
  labels = df['male'].tolist()
  return texts, labels

In [0]:
train_text, train_label = get_texts_and_labels(df_train)
valid_text, valid_label = get_texts_and_labels(df_valid)
test_text, test_label = get_texts_and_labels(df_test)

In [0]:
type(train_text), len(train_text), type(train_text[0]), len(train_text[0]), type(train_text[0][0])

(list, 34146, list, 8, str)

In [0]:
max([len(x) for x in train_text]), max([len(x) for x in valid_text]), max([len(x) for x in test_text])

(47, 42, 42)

In [0]:
# Fit tokenizer on training data
tok = Tokenizer()
tok.fit_on_texts(train_text)
vocab_size = len(tok.word_index) + 1

In [0]:
def get_tokenized_texts(texts):
  encoded_docs = tok.texts_to_sequences(texts)
  padded_docs = pad_sequences(encoded_docs, maxlen=MAXLEN, padding='post')
  return padded_docs

In [0]:
docs_train = get_tokenized_texts(train_text)
docs_valid = get_tokenized_texts(valid_text)
docs_test = get_tokenized_texts(test_text)
print(type(docs_train), len(docs_train), len(docs_valid), len(docs_test))
docs_train[0][:10]

<class 'numpy.ndarray'> 34146 10914 10450


array([  956,     1,  4035,  1154, 13312,     8,    98,   732,     0,
           0], dtype=int32)

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open(glovefile[:-3])
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tok.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [0]:
docs_train.shape, docs_train.dtype

((34146, 50), dtype('int32'))

In [0]:
batchsize = 512

inputs = layers.Input((MAXLEN,), dtype="int32")
raw = layers.Embedding(vocab_size, 
                           EMBEDDING_DIM, 
                           weights=[embedding_matrix], 
                           input_length=MAXLEN, 
                           trainable=False)(inputs)

embed = layers.Dropout(.3)(raw)
lstm1 = layers.LSTM(80, return_sequences=True, go_backwards=True, 
                    dropout=.35, recurrent_dropout=.35)(embed)
dropl = layers.Dropout(.3)(lstm1)
lstm2 = layers.LSTM(80, return_sequences=False, 
                    dropout=.35, recurrent_dropout=.35)(dropl)

lstm1a = layers.LSTM(80, return_sequences=True, 
                     dropout=.35, recurrent_dropout=.35)(embed)
dropla = layers.Dropout(.3)(lstm1a)
lstm2a = layers.LSTM(80, return_sequences=False, go_backwards=True, 
                     dropout=.35, recurrent_dropout=.35)(dropla)

merged = layers.merge.concatenate([lstm2, lstm2a])
dropout0 = layers.Dropout(rate=.5)(merged)
dense = layers.Dense(40, activation='relu')(dropout0)

merged2 = layers.merge.concatenate([merged,dense])
dropout1 = layers.Dropout(rate=.7)(merged2)

pred = layers.Dense(1, activation='sigmoid')(dropout1)

model = Model(inputs=[inputs], outputs=pred)

lr = 1e-3
decay=1e-5
model.compile(optimizer=Adam(lr, decay=decay), loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

model.fit(docs_train, train_label, epochs=2, validation_data=(docs_valid, valid_label), batch_size=batchsize)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 200)      6026200     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 50, 200)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
lstm_1 (LSTM)        

In [0]:
model.layers[0].trainable = True

In [0]:
lr = 2.7e-4
decay = 5.9e-5
model.compile(optimizer=Adam(lr, decay=decay), loss='binary_crossentropy', metrics=['acc'])
model.fit(docs_train, train_label, epochs=60, validation_data=(docs_valid, valid_label), batch_size=batchsize)

Train on 34146 samples, validate on 10914 samples
Epoch 1/60
34146/34146 [==============================] - 29s 845us/step - loss: 0.6710 - acc: 0.5865 - val_loss: 0.6623 - val_acc: 0.5989
Epoch 2/60
34146/34146 [==============================] - 25s 742us/step - loss: 0.6699 - acc: 0.5886 - val_loss: 0.6613 - val_acc: 0.5991
Epoch 3/60
34146/34146 [==============================] - 26s 752us/step - loss: 0.6686 - acc: 0.5913 - val_loss: 0.6603 - val_acc: 0.5995
Epoch 4/60
34146/34146 [==============================] - 26s 760us/step - loss: 0.6671 - acc: 0.5916 - val_loss: 0.6599 - val_acc: 0.6013
Epoch 5/60
34146/34146 [==============================] - 26s 760us/step - loss: 0.6677 - acc: 0.5926 - val_loss: 0.6594 - val_acc: 0.6023
Epoch 6/60
34146/34146 [==============================] - 26s 763us/step - loss: 0.6649 - acc: 0.5933 - val_loss: 0.6593 - val_acc: 0.6011
Epoch 7/60
34146/34146 [==============================] - 26s 759us/step - loss: 0.6677 - acc: 0.5923 - val_loss: 0.